In [1]:
import os
os.chdir("C:\\Users\\erin.melly\\Desktop\\GitHub\\Tax-Calculator")
import behresp
from taxcalc import *
import pandas as pd
import numpy as np

In [2]:
# Reform Stacking
stack0={}
stack1 = {
    "II_brk5": {2021: [416700.00, 416700.00, 208350.00, 416700.00, 416700.00], 
                2026: [502101.0, 502101.0, 251050.0, 502101.0, 502101.0]}, 
    "II_rt5": {2021: 0.33},
    "II_brk6": {2021: [418400.00, 470700.00, 235350.00, 444550.00, 470700.00],
                2026: [504149.0, 567168.0, 283584.0, 535659.0, 567168.0]},
    "II_rt7": {2021: 0.396},
    "PT_brk5": {2021: [416700.00, 416700.00, 208350.00, 416700.00, 416700.00],
                2026: [502101.0, 502101.0, 251050.0, 502101.0, 502101.0]},
    "PT_rt5": {2021: 0.33},
    "PT_brk6": {2021: [418400.00, 470700.00, 235350.00, 444550.00, 470700.00],
                2026: [504149.0, 567168.0, 283584.0, 535659.0, 567168.0]},
    "PT_rt7": {2021: 0.396},
    "ID_ps": {2021: [261500.0, 313800.0, 156900.0, 287650.0, 313800.0],
                2026: [315093.0, 378112.0, 189056.0, 346603.0, 378112.0]}
}
stack2={
    "ID_prt": {2021: 0.03},
    "ID_crt": {2021: 0.8}
}
stack3={
    "PT_qbid_taxinc_thd": {2021: [400000.00, 400000.00, 400000.00, 400000.00, 400000.00],
                            2026: [0.0, 0.0, 0.0, 0.0, 0.0]},
    "PT_qbid_taxinc_gap": {2021: [100000.0, 100000.0, 100000.0, 100000.0, 100000.00],
                            2026: [1.0, 1.0, 1.0, 1.0, 1.0]}
}
stack4={
    "CG_brk4": {2021: [1000000, 1000000, 1000000, 1000000, 1000000]},
    "CG_rt3": {2021: 0.396},
    "CG_TaxAtDeath": {2021: True},
    "CG_exclusion_thd": {2021: [100000, 200000, 100000, 100000, 200000]}
}
stack5 ={
    "EITC_MaxEligAge": {2021: 125}
}
stack6={
    "ID_BenefitSurtax_trt": {2021: 1.0},
    "ID_BenefitSurtax_crt": {2021: 0.28},
    "ID_BenefitSurtax_Switch": {2021: [True, True, False, True, True, True, True]}
}
stack7={
    "SS_Earnings_thd": {2021: 400000}
}
stack8={
    "ALD_IRAContributions_hc": {2021: 1.0},
    "ALD_KEOGH_SEP_hc": {2021: 1.0},
    "IRADC_credit_rt": {2021: 0.26}
}
stack9={
    "CR_ResidentialEnergy_hc": {2021: 0.0}
}

In [3]:
# Elasticity Assumpitions
cg= {2021: -4.184,
     2022: -3.488,
     2023: -2.76,
     2024: -2.76,
     2025: -2.76,
     2026: -2.76,
     2027: -2.76,
     2028: -2.76,
     2029: -2.76,
     2030: -2.76
    }

In [4]:
# Create Reform Lists, Set Records, Create Index Labels
refs_biden = [stack1, stack2, stack3, stack4, stack5, stack6, stack7, stack8, stack9]
refs_base = [stack0, stack1, stack2, stack3, stack4, stack5, stack6, stack7, stack8]
rec = Records()
df = pd.DataFrame(index=["Restore pre-TCJA rates and brackets above $400,000 of income",
                         "Restore limitation on itemized deductions above $400,000 of income (Pease limitation)",
                         "Phase-out Section 199A Deduction for taxpayers above $400,000",
                         "Tax capital gains & dividends at ordinary income rates above $1 million of income & tax at death",
                         "Extend the EITC to childless workers age 65 and older",
                         "Limit tax benefit of itemized deductions to 28 percent of value",
                         "Apply 12.4 percent OASDI payroll tax to eranings above $400,000",
                         "Replace IRA/DC pension contribution deductibility with 26% refundable credit",
                         "Reinstate energy efficiency credit"])

In [5]:
for year in range(2021, 2031):
    pol_base = Policy()
    pol_ref = Policy()
    col=[] 
    response_elasticities = {'sub': 0.25, 'cg': cg[year]}
    for (ref_biden, ref_base) in zip(refs_biden, refs_base):
        pol_base.implement_reform(ref_base)
        calc_base=Calculator(pol_base, rec)
        calc_base.advance_to_year(year)
        pol_ref.implement_reform(ref_biden)
        calc_ref=Calculator(pol_ref, rec)
        calc_ref.advance_to_year(year)
        calc_ref.calc_all()
        base_df, ref_df = behresp.response(calc_base, calc_ref, response_elasticities)
        base_rev= (base_df['combined'] * base_df['s006']).sum()/1e9
        ref_rev = (ref_df['combined'] * ref_df['s006']).sum()/1e9
        dif_rev = ref_rev - base_rev
        col.append(round(dif_rev,2))
    df[year] = col

In [6]:
#Table 2: Revenue Table
df_final = df.append(df.sum().rename('Total'))
df_final

,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
"Restore pre-TCJA rates and brackets above $400,000 of income",18.25,19.02,19.82,20.76,21.71,0.00,0.00,0.00,0.00,0.00
"Restore limitation on itemized deductions above $400,000 of income (Pease limitation)",7.98,8.74,9.30,9.57,10.04,0.00,0.00,0.00,0.00,0.00
"Phase-out Section 199A Deduction for taxpayers above $400,000",38.59,40.16,42.40,44.62,46.56,0.00,0.00,0.00,0.00,0.00
Tax capital gains & dividends at ordinary income rates above $1 million of income & tax at death,-1.64,20.81,46.21,49.65,52.14,38.04,40.09,42.21,44.38,46.67
Extend the EITC to childless workers age 65 and older,-0.52,-0.54,-0.56,-0.58,-0.61,-0.63,-0.66,-0.67,-0.69,-0.70
Limit tax benefit of itemized deductions to 28 percent of value,19.11,19.07,19.06,19.34,19.84,31.21,32.52,33.71,35.18,36.84
"Apply 12.4 percent OASDI payroll tax to eranings above $400,000",58.59,62.47,66.41,70.66,74.99,81.49,86.85,92.74,98.61,104.47
Replace IRA/DC pension contribution deductibility with 26% refundable credit,5.11,5.10,5.29,4.42,5.00,2.04,2.92,3.00,3.22,2.38
Reinstate energy efficiency credit,0.00,-2.44,-2.54,-2.66,-2.77,-2.91,-3.04,-3.17,-3.29,-3.43
Total,145.47,172.39,205.39,215.78,226.90,149.24,158.68,167.82,177.41,186.23
